In [ ]:
from zipfile import ZipFile
with ZipFile("main_dataset_folder.zip", "r") as zip_ref:
  zip_ref.extractall("./")
print("Extraction of zip file complete")

In [ ]:
from datasets import  load_dataset, Audio

dataset = load_dataset(
    "csv",
    data_files="/content/main_dataset_folder/main_dataset.csv",
    split = "train"
)

dataset